In [ ]:
import ultralytics
ultralytics.checks()

import torch
print(torch.backends.mps.is_available())

import os
os.getcwd()

import time

In [ ]:
import torch
print(torch.cuda.is_available())

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="1SxC607xMtZgUVxVWOiL")
project = rf.workspace("aifarms-zlefy").project("object-detection-horseradish-2")
version = project.version(2)
dataset = version.download("yolov8")

In [ ]:
!pip install -U ultralytics

In [ ]:
from ultralytics import YOLO

# best chosen YOLO model
model = YOLO("Horseradish_8n_200epochs.pt")  

# search space
search_space = {
    # "lr0": [1e-5, 1e-1],        # initial learning rate
    "batch_size": [2, 4, 8, 16],
    "epochs": [100, 150, 200],
    "optimizer": ['SGD', 'AdamW']
}
 
start = time.time()

data_path = "/home/sunoj/Documents/Abhi_paper/filtered_dataset/data.yaml"

# Tuning the model
model.tune(
    data=data_path,
    # lr0 = 0.00975,
    degrees = 0.0,
    iterations=10,
    space=search_space,
    plots=False,
    save=False,
    val=False,
    device = 'cuda'
)

end = time.time()

print(f"Time taken for tuning: {end - start} seconds")

## Tuning on Nov 6, 2025

In [ ]:
from ultralytics import YOLO

# Initialize the YOLO model
model = YOLO("/home/sunoj/Documents/Abhi_paper/Horseradish_8n_200epochs.pt")

start = time.time()

search_space = {
    "lr0": (1e-5, 1e-1),
    "momentum": (0.6, 0.99),
    "weight_decay": (0.0, 0.001),
    # "batch_size": (2, 4, 8, 16),
    # "degrees": (0.0, 45.0),
}

data_path = "/home/sunoj/Documents/Abhi_paper/filtered_dataset/data.yaml"

# Tune hyperparameters on horseradish dataset for 50 epochs
model.tune(data=data_path, 
           epochs=50,
           space=search_space, 
           iterations=100, 
           optimizer="AdamW", 
           plots=False, 
           save=False, 
           val=False
           )

end = time.time()

print(f"Time taken for tuning: {end - start} seconds")

## Tuning on Nov 8, 2025 - Optuna search

In [ ]:
# !pip install optuna

In [1]:
import optuna
from ultralytics import YOLO

# your dataset yaml OR data dict
DATA = "filtered_dataset/data.yaml"   # replace with your dataset yaml

# your trained weights
WEIGHTS = "Horseradish_8n_200epochs.pt"


def objective(trial):

    # ----- suggested hyperparameters -----
    lr0           = trial.suggest_float("lr0", 1e-4, 1e-1, log=True)
    lrf           = trial.suggest_float("lrf", 0.01, 0.3)
    momentum      = trial.suggest_float("momentum", 0.80, 0.97)
    weight_decay  = trial.suggest_float("weight_decay", 5e-5, 1e-3, log=True)
    warmup_epochs = trial.suggest_int("warmup_epochs", 0, 3)

    box = trial.suggest_float("box", 3.0, 10.0)
    cls = trial.suggest_float("cls", 0.2, 2.0)
    dfl = trial.suggest_float("dfl", 0.2, 3.0)

    # translate = trial.suggest_float("translate", 0.0, 0.2)
    # scale     = trial.suggest_float("scale", 0.5, 1.5)

    optimizer = trial.suggest_categorical("optimizer", ["SGD", "Adam", "AdamW"])


    # ----- load base model -----
    model = YOLO(WEIGHTS)

    # ----- train: short schedule for speed -----
    result = model.train(
        data=DATA,
        epochs=30,       # keep small for tuning
        lr0=lr0,
        # lrf=lrf,
        # momentum=momentum,
        weight_decay=weight_decay,
        # warmup_epochs=warmup_epochs,
        # box=box,
        # cls=cls,
        # dfl=dfl,
        # translate=translate,
        # scale=scale,
        optimizer=optimizer,
        verbose=False
    )

    # get metrics dictionary
    metrics = result.results_dict

    # segmentation: use metrics/seg_50
    if "metrics/seg_50" in metrics:
        score = metrics["metrics/seg_50"]

    # detection: metrics/mAP50(B)
    elif "metrics/mAP50(B)" in metrics:
        score = metrics["metrics/mAP50(B)"]

    else:
        # if missing, penalize
        score = 0

    return score


# create study
study = optuna.create_study(direction="maximize")

# run optimization
study.optimize(objective, n_trials=50)   # increase trials if needed

# print result
print("\nbest score:", study.best_value)
print("best params:", study.best_params)


/home/sunoj/miniconda3/envs/cvml/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-11-09 13:17:39,186] A new study created in memory with name: no-name-d8ec0839-9a70-408e-b264-ae710d9e6eea


New https://pypi.org/project/ultralytics/8.3.226 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5060 Ti, 7672MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=filtered_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.006228103875274696, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=Horseradish_8n_200epochs.pt, momentum=0.937, mosaic=1.0, multi_scale=

[I 2025-11-09 13:19:13,997] Trial 0 finished with value: 0.9026054893411402 and parameters: {'lr0': 0.006228103875274696, 'lrf': 0.29632659975475584, 'momentum': 0.9271927280047216, 'weight_decay': 0.00018482798984093793, 'warmup_epochs': 3, 'box': 4.1959389951980635, 'cls': 1.193042686586487, 'dfl': 2.7162064618948496, 'optimizer': 'AdamW'}. Best is trial 0 with value: 0.9026054893411402.


New https://pypi.org/project/ultralytics/8.3.226 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5060 Ti, 7672MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=filtered_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.014443442838813877, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=Horseradish_8n_200epochs.pt, momentum=0.937, mosaic=1.0, multi_scale=

[I 2025-11-09 13:20:46,460] Trial 1 finished with value: 0.9287131329021513 and parameters: {'lr0': 0.014443442838813877, 'lrf': 0.2726188685569015, 'momentum': 0.9239792834764654, 'weight_decay': 0.0002903627635593407, 'warmup_epochs': 2, 'box': 6.654133868571913, 'cls': 0.35003639996884195, 'dfl': 1.5251121547509199, 'optimizer': 'AdamW'}. Best is trial 1 with value: 0.9287131329021513.


...

New https://pypi.org/project/ultralytics/8.3.226 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5060 Ti, 7672MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=filtered_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.011063908932548706, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=Horseradish_8n_200epochs.pt, momentum=0.937, mosaic=1.0, multi_scale=

[I 2025-11-09 14:31:35,525] Trial 48 finished with value: 0.9405465033366837 and parameters: {'lr0': 0.011063908932548706, 'lrf': 0.21903004924904773, 'momentum': 0.8397184639013191, 'weight_decay': 0.00018995047411129656, 'warmup_epochs': 2, 'box': 7.0454607294703075, 'cls': 1.3658247213155048, 'dfl': 0.7092590689312507, 'optimizer': 'SGD'}. Best is trial 45 with value: 0.949476498833146.


New https://pypi.org/project/ultralytics/8.3.226 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5060 Ti, 7672MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=filtered_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.029083283985813755, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=Horseradish_8n_200epochs.pt, momentum=0.937, mosaic=1.0, multi_scale=

[I 2025-11-09 14:33:05,475] Trial 49 finished with value: 0.8956992443764514 and parameters: {'lr0': 0.029083283985813755, 'lrf': 0.17200913981068333, 'momentum': 0.8169000645530617, 'weight_decay': 0.00021761499009447234, 'warmup_epochs': 2, 'box': 6.802213633331742, 'cls': 1.9841549134917298, 'dfl': 1.0571367314303122, 'optimizer': 'SGD'}. Best is trial 45 with value: 0.949476498833146.



best score: 0.949476498833146
best params: {'lr0': 0.013100173741089865, 'lrf': 0.17890342199276, 'momentum': 0.8361091720942576, 'weight_decay': 0.00020253734502285683, 'warmup_epochs': 2, 'box': 7.151807341128154, 'cls': 1.4898428013457137, 'dfl': 0.7678327533386852, 'optimizer': 'SGD'}
